## TODO

- Convergence stuff
- Default values in constants
- rewrite core utilities to be nicer
- get automatic emax
- warning for insulators about the fermi level positioning
- add mesh and converge to batch
- More tests
- Better docs
- dependency on how we choose the perpendicular directions
- Use ReadThe Docs [addons](https://docs.readthedocs.io/en/stable/addons.html)


In [1]:
import numpy as np

import grogupy
import grogupy.viz

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/_tqdm.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# infile = "../benchmarks/Fe_colinear/Fe.fdf"
infile = "../benchmarks/Cr3/Cr3.fdf"
# infile = "../benchmarks/Fe3GeTe2/Fe3GeTe2.fdf"

In [3]:
simulation_kspace = grogupy.Kspace([1, 1, 1])
simulation_kspace.plot().show()
simulation_kspace

<grogupy.Kspace kset=[1 1 1], NK=1>

In [4]:
simulation_contour = grogupy.Contour(
    eset=100,
    esetp=10000,
    emin=None,
    emax=0,
    emin_shift=-5,
    emax_shift=0,
    eigfile=infile,
)
simulation_contour.plot().show()
simulation_contour

<grogupy.Contour emin=-8.074511730000001, emax=0, eset=100, esetp=10000>

In [5]:
simulation_hamiltonian = grogupy.Hamiltonian(
    infile,
    [0, 0, 1],
)

Calculate H_XC: 100%|██████████| 3/3 [00:00<00:00, 11105.84it/s]


In [6]:
xyz = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

simulation = grogupy.Builder(xyz, matlabmode=True)

simulation.add_kspace(simulation_kspace)
simulation.add_contour(simulation_contour)
simulation.add_hamiltonian(simulation_hamiltonian)
simulation.ref_xcf_orientations

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/physics/builder.py:228: UserWarning:

Matlabmode is used, the exchange field reference directions were set to x,y,z!



[{'o': array([1, 0, 0]),
  'vw': [array([0, 1, 0]),
   array([0, 0, 1]),
   array([0.        , 0.70710678, 0.70710678])]},
 {'o': array([0, 1, 0]),
  'vw': [array([1, 0, 0]),
   array([0, 0, 1]),
   array([0.70710678, 0.        , 0.70710678])]},
 {'o': array([0, 0, 1]),
  'vw': [array([1, 0, 0]),
   array([0, 1, 0]),
   array([0.70710678, 0.70710678, 0.        ])]}]

In [7]:
magnetic_entities = [
    dict(atom=0, l=2),
    dict(atom=1, l=2),
    dict(atom=2, l=2),
]
simulation.add_magnetic_entities(magnetic_entities)
simulation.magnetic_entities

Add magnetic entities:: 100%|██████████| 3/3 [00:00<00:00, 1752.01it/s]


[<grogupy.MagneticEntity tag=0Cr(l:2), SBS=20>,
 <grogupy.MagneticEntity tag=1Cr(l:2), SBS=20>,
 <grogupy.MagneticEntity tag=2Cr(l:2), SBS=20>]

In [8]:
pairs = [
    dict(ai=0, aj=1, Ruc=np.array([0, 0, 0])),
    dict(ai=1, aj=2, Ruc=np.array([0, 0, 0])),
    dict(ai=2, aj=0, Ruc=np.array([0, 0, 0])),
]
simulation.add_pairs(pairs)
simulation.pairs

Add pairs:: 100%|██████████| 3/3 [00:00<00:00, 8548.17it/s]


[<grogupy.Pair tag1=0Cr(l:2), tag2=1Cr(l:2), Ruc=[0 0 0]>,
 <grogupy.Pair tag1=1Cr(l:2), tag2=2Cr(l:2), Ruc=[0 0 0]>,
 <grogupy.Pair tag1=2Cr(l:2), tag2=0Cr(l:2), Ruc=[0 0 0]>]

In [9]:
simulation.solve()

Setup perturbations for rotated hamiltonian: 100%|██████████| 3/3 [00:00<00:00, 23431.87it/s]


In [10]:
print(simulation.to_magnopy())

# ================================================================================
# grogupy version: 0.0.9
# Input file: ../benchmarks/Cr3/Cr3.fdf
# Spin mode: SPIN-ORBIT
# ================================================================================
# SLURM job ID: Could not be determined.
# Architecture: CPU
# Number of nodes in the parallel cluster: 1
# Parallelization is over: K
# Solver used for Greens function calculation: Sequential
# Solver used for Exchange tensor: grogupy
# Solver used for Anisotropy tensor: grogupy
# ================================================================================
# Cell [Ang]:
# 1.442498074906033700e+01 -2.498479955557547072e+01 0.000000000000000000e+00
# 1.442498074906033700e+01 2.498479955557547072e+01 0.000000000000000000e+00
# 0.000000000000000000e+00 0.000000000000000000e+00 2.884996149812067401e+01
# ================================================================================
# DFT axis: [0 0 1]
# Quantization axis and perpendi

In [11]:
from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_rotvec(np.pi / 2 * np.array([0, 0, 1]))
r.as_matrix()

array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [12]:
grogupy.save(simulation, "test_builder.pkl", compress=0)
grogupy.save(simulation.contour, "test_contour.pkl", compress=0)
grogupy.save(simulation.kspace, "test_kspace.pkl", compress=0)
grogupy.save(simulation.hamiltonian, "test_hamiltonian.pkl", compress=0)
grogupy.save(simulation.pairs[0], "test_pair.pkl", compress=0)
grogupy.save(simulation.magnetic_entities[0], "test_magnetic_entity.pkl", compress=0)
grogupy.save(simulation.times, "test_default_timer.pkl", compress=0)